Note: No level 1 heading here so that the numbering starting with ''Introduction'' below matches the manuscript.

## Imports

In [ ]:
#%reset -f  # gives "invalid alias" errors
%config InlineBackend.rc = {}
%config InlineBackend.figure_format = "retina"
%matplotlib inline

%load_ext autoreload
%autoreload 2

import io
import pickle
from pprint import pprint

import matplotlib.dates as mpl_d
import matplotlib.pyplot as plt
import matplotlib.ticker as mpl_t
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.linear_model

import IPython.display

import experiment as xp
import google_health_trends as ght
import ili
import models
import noise
import wikipedia as wp
import u

np.random.seed(u.random_seed)

In [ ]:
def plt_timeseries_setup(ax=None):
   if (ax is None):
      ax = plt.gca()
   ax.set_xlim("2011-Jun-5","2016-Jul-24")
   ax.xaxis.set_major_locator(mpl_d.YearLocator())
   #ax.tick_params("x", direction="in", length=10.0)
   ax.set_xticklabels(ax.get_xticks(), ha="left")
   #ax.xaxis.set_major_formatter(mpl_d.DateFormatter("%Y-%b-%d"))
   ax.xaxis.set_major_formatter(mpl_d.DateFormatter("%Y"))
   ax.xaxis.set_minor_locator(mpl_d.MonthLocator(bymonth=(3,5,7,9,11)))

In [ ]:
sns.palplot(u.colors)

## Load data

In [ ]:
ili.load()
wp.load()
ght.load()
with open(u.datapath_tl / "out.pickle", "rb") as fp:
   out = pickle.load(fp)

## Sanity check of output contents

### Top level dictionary

In [ ]:
pprint(sorted(out.keys()))
assert (out.keys() == {"decept",
                       "features",
                       "lambda_cv_means_bests",
                       "lambda_cv_means_bias",
                       "lambda_cv_selected",
                       "models",
                       "results",
                       "synthetic_draws"})

### features

Input features themselves. Dictionary.

* keys: name of feature class
* values: DataFrame of all features of that type.

In [ ]:
pprint([(k, len(v.columns), len(v.index), type(v))
        for k, v in out["features"].items()])

### decept

Deceptiveness of each feature. Nested dictionary.

* keys:
  * name of feature class
  * deceptiveness noisiness
* values: Series, deceptiveness of each feature in that class.

In [ ]:
for (fn, fn_d) in out["decept"].items():
   for (dn, s) in fn_d.items():
      print("(%s, %0.2f): %d, %s" % (fn, dn, len(s.index), type(s)))

### lambda cross-validation

List of best lambdas:

In [ ]:
if (out["lambda_cv_means_bests"] is not None):
   print(type(out["lambda_cv_means_bests"]), len(out["lambda_cv_means_bests"]))

Bias function added to each model's lambda RMSE:

In [ ]:
if (out["lambda_cv_means_bias"] is not None):
   out["lambda_cv_means_bias"].plot(lw=1, logx=True); None

Selected lambda (mean of the best for each model):

In [ ]:
(xp.lambda_cv_min, xp.lambda_cv_max, xp.lambda_cv_step_ct)

In [ ]:
out["lambda_cv_selected"]

### models

All the models we fitted. Nested dictionary.

* keys:
  * name of input feature class
  * training start
  * deceptiveness noise
  * model class name
* values: model objects

Check consistency. Features and deceptiveness in models should be references, not copies.

In [ ]:
assert set(xp.f_feature_name) == out["models"].keys()
for (fn, fn_d) in out["models"].items():
   assert set(xp.f_training_start) == fn_d.keys()
   for (ts, ts_d) in fn_d.items():
      assert set(xp.f_decept_noise) == ts_d.keys()
      for (dn, dn_d) in ts_d.items():
         assert {mc.name for mc in xp.f_model_class} == dn_d.keys()
         for (mn, m) in dn_d.items():
            assert fn == m.features_name
            assert ts == m.training_start
            assert dn == m.decept_noise
            assert mn == m.name
            assert id(m.features) == id(out["features"][m.features_name])
            assert id(m.decept) == id(out["decept"][m.features_name][dn])

### results

In [ ]:
assert {ef.__name__ for ef in xp.e_functions} == out["results"].keys()
pprint(sorted(out["results"].keys()))

Check rows and columns of results dataframes.

In [ ]:
for df in out["results"].values():
   assert set(df.columns.get_level_values(0)) == set(xp.f_feature_name)
   assert (   set(df.columns.get_level_values(1))
           == set(mc.name for mc in xp.f_model_class))


### synthetic_draws

This is the raw return values from `features_draw()` for the synthetic features.

In [ ]:
assert type(out["synthetic_draws"]) == list
for i in out["synthetic_draws"]:
   assert len(i) == 8

## Re-distributable experiment dataset

I believe everything except the Google data can be redistributed. For now, just put the synthetic features in an Excel spreadsheet.

In [ ]:
out["features"]["synthetic"].head()

In [ ]:
out["features"]["synthetic"].to_excel("synthetic.xlsx")

**If we later want to scrub `out.pickle` instead**:
2. change references to None
3. grep around in the blob to make sure we didn't miss any references (e.g. with `pickletools` module)

# Introduction

In [ ]:
dfeat = ili.us_national.copy() \
        + 0.30 * noise.normal(ili.us_national.index) \
        + 8.00 * noise.linear(ili.us_national.index, "2014-07-06")

plt.figure()

plt.plot(ili.us_national, color="gray", lw=2.0,
         label="ILI")
plt.plot(dfeat, color=u.colors[0], lw=1.0,
         label="model")

plt.ylabel("ILI %")
plt.ylim(0,11.5)

plt_timeseries_setup()
plt.legend()
plt.savefig("us_ili_synthetic_deceptive.pdf")

In [ ]:
mq = pd.read_csv(u.datapath_tl / "misc.ght.csv",
                 header=0, skiprows=[1,], index_col=0, parse_dates=[0])

In [ ]:
plt.figure()

#plt.plot(ili.us_national, color="black", lw=1.5, label="ILI")

flu3 = ili.us_national[:"2014-Jun-29"]

labeled=False
for qs in ("how long are you contagious",
           "flu contagious",
           "how long am i contagious",
           "influenza contagious",
           "when are you contagious"):
   query = mq[qs]
   query3 = query[:"2014-Jun-29"]
   reg = sklearn.linear_model.LinearRegression()
   # .values.reshape() is to avoid two different deprecation warnings.
   reg.fit(query3.values.reshape(-1,1), flu3)
   query_normalized = reg.coef_[0] * query + reg.intercept_
   if (labeled):
      label="_nolabel_"  # https://stackoverflow.com/a/35710894
   else:
      label="contagiousness searches"
      labeled=True
   plt.plot(query_normalized, lw=0.5, color=u.colors[0], label=label)

plt.plot(ili.us_national, color="gray", lw=2.0, label="ILI")
plt.axvline(x=u.train_end, color="black", lw=0.5)

plt.ylabel("ILI %")

plt.legend()
plt_timeseries_setup()
plt.savefig("us_ili_and_deceptive.pdf")

# Methods

## Data sources

### U.S. influenza-like illness

In [ ]:
plt.figure()

plt.plot(ili.us_national, color="gray", lw=2.0)

plt.ylabel("ILI %")

plt_timeseries_setup()
plt.savefig("us_ili.pdf")

### Flu-related concepts

(No figures in this section.)

### Web search volume

#### Example features

In [ ]:
plt.figure()
y1 = plt.axes()
#y2 = plt.twinx()

#y2.plot(ili.us_national, color="gray", lw=1.5)
y1.plot(ght.topics["Influenza A virus (Virus)"], color=u.colors[2], lw=1.5,
        label="topic: Influenza A virus (Virus)")
y1.plot(ght.raws["respiratory system"], color=u.colors[0], lw=1.5,
        label="raw query: respiratory system")

y1.set_ylabel("searches per million")
y1.set_ylim([0,38.5e-6])
y1.yaxis.set_major_formatter(mpl_t.FuncFormatter(lambda y, pos: "%g" % (y*1e6)))

plt_timeseries_setup()
plt.legend(bbox_to_anchor=(0.48,0.68), labelspacing=0.3)
plt.savefig("ght_examples.pdf")

#### Table of category distance and corresponding deceptiveness:

In [ ]:
[(i, ght.deceptiveness(i)) for i in range(1,8)]

In [ ]:
len(ght.raw_decept.index)

In [ ]:
len(ght.topic_decept.index)

In [ ]:
ght.raw_decept.head(10)

In [ ]:
ght_raw_dist_hist = ght.raw_decept.groupby(["distance"])["distance"].count()
ght_raw_dist_hist.name = "raw"
ght_raw_dist_hist

In [ ]:
ght_raw_decept_hist = ght.raw_decept.groupby(["decept"])["decept"].count()
ght_raw_decept_hist.name = "raw"
ght_raw_decept_hist

In [ ]:
ght.topic_decept.head(10)

In [ ]:
ght_topic_dist_hist = ght.topic_decept.groupby(["distance"])["distance"].count()
ght_topic_dist_hist.name = "topic"
ght_topic_dist_hist

In [ ]:
ght_topic_decept_hist = ght.topic_decept.groupby(["decept"])["decept"].count()
ght_topic_decept_hist.name = "topic"
ght_topic_decept_hist

In [ ]:
ght_dist_hist = pd.concat([ght_raw_dist_hist, ght_topic_dist_hist],
                          axis=1)
ght_dist_hist

In [ ]:
ght_decept_hist = pd.concat([ght_raw_decept_hist, ght_topic_decept_hist],
                            axis=1)
ght_decept_hist

In [ ]:
buf = io.StringIO()
buf.write("""\
\\begin{tabular}{ccrr}
\\toprule
  \\textbf{Distance}
& \\textbf{Deceptiveness}
& \\textbf{Strings}
& \\textbf{Topics} \\\\
\\cmidrule(r){1-2}
\\cmidrule(l){3-4}
""")

for r in ght_dist_hist.itertuples():
   buf.write("%d & %0.02f & %d & %d \\\\ \n"
             % (r.Index, ght.deceptiveness(r.Index), r.raw, r.topic))

buf.write("""\
\\bottomrule
\\end{tabular}
""")

print(buf.getvalue())
with open("ght_decept.tex", "w") as fp:
   fp.write(buf.getvalue())

### Synthetic input features

#### Basis functions

In [ ]:
plt.figure(figsize=(u.fig_width, 1.4*u.fig_width))

def subplot_setup(ax, xlabels=False):
   plt_timeseries_setup(ax)
   ax.set_ylim(-0.15,1.15)
   ax.yaxis.set_major_locator(mpl_t.FixedLocator([0,1]))
   if (not xlabels):
      ax.tick_params("x", labelbottom=False)

def subplot(loc, key, title=None, xlabels=False):
   ax = plt.subplot(loc)
   if (title is not None):
      plt.title(title)
   plt.plot(ili.us_national / ili.us_national.max(), color="gray", lw=0.5)
   plt.plot(xp.systematic_bases[key])
   subplot_setup(ax, xlabels)
      
subplot(711, "oprah annual", "Oprah effects: annual, fore, late")
subplot(712, "oprah fore")
subplot(713, "oprah late", xlabels=True)

subplot(714, "drift steady", "Drift: steady, late")
subplot(715, "drift late", xlabels=True)

subplot(716, "cycle annual", "Cycle: annual, ending")
subplot(717, "cycle ending", xlabels=True)

# FIXME: This doesn't actually yield variable spacing.
# Maybe GridSpec or mpl_toolkits.axes_grid1?
plt.tight_layout(pad=0, h_pad=0, w_pad=0)
plt.savefig("bases.pdf")

#### Features

In [ ]:
def fmt_weights(w):
   s = (  "%.2f / %.2f / %.2f (%.2f %.2f %.2f; %.2f %.2f; %.2f %.2f)"
        % ((w[0], w[1], w[2:].sum()) + tuple(w[2:])))
   return s.replace("0.00", "0")

def plot_feature(f):
   time_series = f[0]
   weights = f[7]
   plt.figure(figsize=(u.fig_width, 0.2*u.fig_width))
   ax = plt.axes()
   plt.title("%s: %s" % (time_series.name, fmt_weights(weights)))
   plt.plot(models.normalize(ili.us_national), color="gray", lw=0.5)
   plt.plot(models.normalize(time_series), lw=1)
   #ax.set_ylim(-0.05,1.05)
   plt.tick_params("x", bottom="off", labelbottom=False)
   #plt.tick_params("y", left="off", labelleft=False)
   plt.show()

for f in sorted(out["synthetic_draws"], key=lambda f: f[7][1:].sum())[:1000]:
   plot_feature(f)

In [ ]:
plt.figure()

plt.plot(out["features"]["synthetic"][314], lw=1.5, label="least deceptive")
plt.plot(out["features"]["synthetic"][386], lw=1.5, label="middling")
plt.plot(out["features"]["synthetic"][412], lw=1.5, label="most deceptive")

plt.yticks([])

plt_timeseries_setup()
plt.legend() #bbox_to_anchor=(0.48,0.68), labelspacing=0.3)
plt.savefig("synthetic_examples.pdf")

#### Histogram of deceptiveness

We compute the histogram rather than plotting it directly so that we can put both types of features on the same plot.

In [ ]:
bins = np.linspace(0, 1, 11)
rounded = pd.cut(out["decept"]["synthetic"][0.0], bins=bins, right=False,
                 labels=(bins[:-1]))
syn_hist = rounded.value_counts()
syn_hist.sort_index(inplace=True)
syn_hist

In [ ]:
ght_decept_hist

In [ ]:
plt.figure()

plt.plot(ght_decept_hist["raw"], "o-", label="query strings",
         lw=1.0, ms=6, color=u.colors[0])
plt.plot(ght_decept_hist["topic"], "o-", label="topics",
         lw=1.0, ms=6, color=u.colors[1])
plt.bar(list(syn_hist.index), list(syn_hist), label="synthetic",
        width=0.1, align="edge",
        color=u.colors[4], linewidth=0.0, edgecolor="black")

plt.xlabel("deceptiveness")
plt.xlim([-0.02,1.02])
plt.ylabel("number of features")
plt.ylim([0,200])

plt.legend(loc="upper left")
plt.savefig("decept_hist.pdf")

# Results

## Tinkering for us

### Plot all the models

This is just for our reference, not the paper. We'll do better below.

In [ ]:
for (fn, fn_d) in out["models"].items():
   plt.figure(figsize=(7,30))
   plt.suptitle(fn, size=16)
   sp_i = 0
   for (ts_i, ts) in enumerate(fn_d.keys()):
      ts_d = fn_d[ts]
      for (dn_i, dn) in enumerate(ts_d.keys()):
         dn_d = ts_d[dn]
         sp_i += 1
         plt.subplot(len(xp.f_decept_noise) * len(xp.f_training_start), 1, sp_i)
         plt.title("ts %d / dn %g" % (ts, dn))
         plt.xticks([])
         plt.yticks([])
         plt.plot(ili.us_national, label="__nolabel__",
                  color="lightgray", lw=3.0)
         for m in dn_d.values():
            if (m.name in ("SMC", "SMM", "SMR")): continue
            plt.plot(m.prediction, label=m.name, lw=1.0)
         plt.axvline(x=u.train_end, color="black", lw=0.5)
         plt.legend()
   
   plt.tight_layout(pad=0, h_pad=1, w_pad=1)
   plt.subplots_adjust(top=0.963)
   plt.savefig("all.%s.pdf" % fn)

### Scatter plot of deceptivness vs. correlation during training

Scatter plot of synthetic feature correlation during training period with ILI vs. deceptiveness. If deceptiveness is predicted well by correlation, then we don't need it, i.e., we probably sampled the synthetic features wrong.

In [ ]:
plt.figure(figsize=(u.fig_width, u.fig_width))

x = [ili.us_national_train.corr(f)
     for (_, f) in out["features"]["synthetic"].iteritems()]
y = out["decept"]["synthetic"][0]

plt.scatter(x, y)

plt.xlabel("correlation during training")
#plt.xlim(0,1)
plt.ylabel("feature deceptiveness")
#plt.ylim(0,1)
None

## All results in big tables

In [ ]:
for (m, df) in out["results"].items():
   print(m)
   IPython.display.display(df)

## 𝝀

In [ ]:
out["lambda_cv_selected"]

### Histogram of best 𝝀 selections

In [ ]:
plt.hist(out["lambda_cv_means_bests"], bins=100, log=True)
None

In [ ]:
h = pd.Series(out["lambda_cv_means_bests"]).value_counts()
h.sort_index(inplace=True)
h

### Each model’s 𝝀 plot

In [ ]:
for (fn, fn_d) in out["models"].items():
   plt.figure(figsize=(7,30))
   plt.suptitle(fn, size=16)
   sp_i = 0
   for (ts_i, ts) in enumerate(fn_d.keys()):
      ts_d = fn_d[ts]
      for (dn_i, dn) in enumerate(ts_d.keys()):
         dn_d = ts_d[dn]
         sp_i += 1
         plt.subplot(len(xp.f_decept_noise) * len(xp.f_training_start), 1, sp_i)
         plt.title("ts %d / dn %g" % (ts, dn))
         for m in dn_d.values():
            if (m.name in ("SMC", "SMM", "SMR")): continue
            plt.semilogx(m.lambda_cv_means, label=m.name, lw=1.0)
         plt.axvline(x=out["lambda_cv_selected"], color="black", lw=0.5)
         plt.ylim(0,1)
         plt.legend()
   
   plt.tight_layout(pad=0, h_pad=1, w_pad=1)
   plt.subplots_adjust(top=0.963)
   plt.savefig("lambda.%s.pdf" % fn)

## Selected models

In [ ]:
plt.figure(figsize=(u.fig_width, 1.1*u.fig_width))

for (i, fn) in enumerate(xp.f_feature_name):
   plt.subplot(3, 1, i+1)
   plt.title({'synthetic': 'synthetic',
              'raw':       'query strings',
              'topic':     'topics'}[fn])
   
   plt.plot(ili.us_national, label="ILI", color="lightgray", lw=4.0)

   models_ = out["models"][fn][0][0]
   for m in models_.values():
      plt.plot(m.prediction, label=m.name, lw=1.0)
   
   plt_timeseries_setup()
   plt.ylim(0, 8.0)
   if (i == 1):
      plt.ylabel("ILI %")
   if (i != 2):
      plt.xticks([])
   plt.axvline(x=u.train_end, color="black", lw=0.5)
   if (i == 0):
      plt.legend()

plt.tight_layout(h_pad=1)
plt.savefig("selected_models.pdf")

## Main results figure

This figure has nine subfigures. The various factors and metrics are reported as follows:

* metrics: one figure per (metric, season pair)
* season: see above
* feature class: subplot rows
* training start: subplot columns
* decept noise: X axis
* metric value: Y axis
* model class: line style

In [ ]:
# These limits are all set so lower is better.
limits = { ("rmse",               "synthetic"): ( 0.00,  3.00),
           ("rmse",               "raw"):       ( 0.00,  0.90),
           ("rmse",               "topic"):     ( 0.00,  0.90),
           ("r2",                 "synthetic"): ( 1.00,  0.85),
           ("r2",                 "raw"):       ( 1.00,  0.75),
           ("r2",                 "topic"):     ( 1.00,  0.75),
           ("peak_timing_abs",    "synthetic"): (-0.50, 12.50),
           ("peak_timing_abs",    "raw"):       (-0.50, 12.50),
           ("peak_timing_abs",    "topic"):     (-0.50, 12.50),
           ("peak_intensity_abs", "synthetic"): ( 0.00,  2.10),
           ("peak_intensity_abs", "raw"):       ( 0.00,  2.10),
           ("peak_intensity_abs", "topic"):     ( 0.00,  2.10),
           ("hit_rate",           "synthetic"): ( 1.00,  0.00),
           ("hit_rate",           "raw"):       ( 1.00,  0.00),
           ("hit_rate",           "topic"):     ( 1.00,  0.00) }

rowheads = { "synthetic": "synthetic",
             "raw": "query strings",
             "topic": "topics" }

def results_plot(metric, season):
   
   plt.figure(figsize=(u.fig_width, 0.9*u.fig_width))

   sp_row_ct = len(xp.f_feature_name)
   sp_col_ct = len(xp.f_training_start)
   for (sp_row, fc) in enumerate(xp.f_feature_name):
      for (sp_col, ts) in enumerate(xp.f_training_start):
         plt.subplot(sp_row_ct, sp_col_ct, sp_col_ct * sp_row + sp_col + 1)
         for (j, mc) in enumerate(i.name for i in xp.f_model_class):
            # This garbage is the best indexing syntax I could come up with.
            s = out["results"][metric][fc,mc] \
                   .xs(season, level="test season", axis=0) \
                   .xs(ts, level="training start", axis=0)
            if (mc == "ridge"):
               plt.plot(s, "-",  lw=3, color="lightgray", label=mc)
            else:
               plt.plot(s, "o-", lw=1, ms=3, color=u.colors[j], label=mc)
         plt.tick_params(length=3, direction="out")
         plt.xlim(-0.03, 1.03)
         if ((metric,fc) in limits):
            plt.ylim(limits[(metric,fc)])
         if (sp_row == 0):
            titles = ["training: 3 seasons\n",
                      "2 seasons\n%s" % rowheads[fc],
                      "1 season\n"]
            if (sp_col == 0):
               plt.legend(loc="upper left", fontsize="small",
                          borderpad=0.2, labelspacing=0.3)
         else:
            titles = ["", rowheads[fc], ""]
         plt.title(titles[ts], verticalalignment="bottom")
         if (sp_row == 2):
            plt.xticks(xp.f_decept_noise, ["0", "", "0.15", "0.4", "1"])
            if (sp_col == 1):
               plt.xlabel("deceptiveness noise added")
         else:
            plt.xticks(xp.f_decept_noise, ("" for i in xp.f_decept_noise))
         if (sp_col == 0):
            if (sp_row == 1):
               plt.ylabel("%s : %s season"
                          % (metric.upper(), ["4th", "5th"][season-3]))
         else:
            plt.tick_params(axis="y", labelleft="off")

   #plt.tight_layout(pad=0, h_pad=0, w_pad=0)
   plt.tight_layout(w_pad=1)
   plt.savefig("results_%s_%d.pdf" % (metric, season))

for metric in reversed(list(out["results"].keys())):
   for season in (4, 3):
      results_plot(metric, season)

## Summary

### All data points

Each column corresponds to one box in the box plot below. So, we should have 12 columns.

rows:
  - metric (RMSE, $r^2$, hit rate)
  - deceptiveness noise (just 0, 0.05)
  - training start
  - test season

columns:
  - feature class
  - algorithm, excluding ridge
  
elements:
  - improvement over ridge (e.g. if it's twice as good, 2.0)

In [ ]:
out["results"]["hit_rate"]["synthetic","linear fridge"][0,3,0]

In [ ]:
metrics = ["rmse", "r2", "hit_rate"]
dns = [0, 0.05]
model_names = [i.name for i in xp.f_model_class if i != models.Ridge]
# "raw/topic" is the raw results normalized against topic's ridge
feature_names = ["synthetic", "raw", "raw/topic", "topic"]

index = pd.MultiIndex.from_product([metrics, dns,
                                    xp.f_training_start, xp.e_seasons],
                                   names=["metric", "decept noise",
                                          "training start", "season"])
columns = pd.MultiIndex.from_product([feature_names,model_names],
                                     names=["features", "model"])
imp = pd.DataFrame(index=index, columns=columns, dtype=np.float64)

# There is probably a fancy Pandas way to do this, but I'm tired and I
# understand this loop.
for metric in metrics:
   for dn in dns:
      for ts in xp.f_training_start:
         for season in xp.e_seasons:
            for fc in feature_names:
               if (fc == "raw/topic"):
                  n_fc = "raw"
                  d_fc = "topic"
               else:
                  n_fc = fc
                  d_fc = fc
               ridge_perf = out["results"][metric][d_fc,"ridge"][dn,season,ts]
               for mn in model_names:
                  perf = out["results"][metric][n_fc,mn][dn,season,ts]
                  if (metric in ("r2", "hit_rate")):
                     # Measure of correlation: higher is better.
                     # Compute the reduction in distance to 1 (perfect).
                     ratio = (1-ridge_perf) / (1-perf)
                  else:
                     assert (metric in metrics)
                     # Measure of error: lower is better.
                     # Compute the reduction in error.
                     ratio = ridge_perf / perf
                  imp[fc,mn][metric,dn,ts,season] = ratio

imp

### RMSE summaries for body text

In [ ]:
rmse = out["results"]["rmse"]

In [ ]:
# Ridge is same for every noise level, so just use 0.
ridge = rmse.loc[0.00, (slice(None), "ridge")]
ridge

In [ ]:
np.median(ridge.values), np.min(ridge.values), np.max(ridge.values)

In [ ]:
rmse.columns

In [ ]:
gridge_low = rmse.loc[0:0.05, (slice(None), ("threshold fridge",
                                             "linear fridge",
                                             "quadratic fridge",
                                             "quartic fridge"))]
gridge_low

In [ ]:
(np.median(gridge_low.values),
 np.min(gridge_low.values),
 np.max(gridge_low.values))

In [ ]:
gridge_high = rmse.loc[0.4:1.0, (slice(None), ("threshold fridge",
                                               "linear fridge",
                                               "quadratic fridge",
                                               "quartic fridge"))]
gridge_high

In [ ]:
(np.median(gridge_high.values),
 np.min(gridge_high.values),
 np.max(gridge_high.values))

### Table

rows:
  - feature class
  - algorithm
  
columns:
  - max, median, min

In [ ]:
min_df = imp.min()
median_df = imp.median()
max_df = imp.max()

In [ ]:
model_names = [i.name for i in xp.f_model_class if i != models.Ridge]

index = pd.MultiIndex.from_product([xp.f_feature_name, model_names],
                                   names=["features", "model"])
columns = pd.MultiIndex.from_product([["min", "median", "max"]],
                                     names=["aggregate"])
summary = pd.DataFrame(index=index, columns=columns, dtype=np.float64)

summary["min"] = min_df
summary["median"] = median_df
summary["max"] = max_df

summary

### Box plot

In [ ]:
positions = { "threshold fridge": [ 1, 6,11,16],
              "linear fridge":    [ 2, 7,12,17],
              "quadratic fridge": [ 3, 8,13,18],
              "quartic fridge":   [ 4, 9,14,19] }

plt.figure(figsize=(u.fig_width, 1.3*u.fig_width))

for (metric_i, metric) in enumerate(["rmse", "r2", "hit_rate"]): 

   plt.subplot(3, 1, metric_i + 1)
   plt.title(metric)
   
   for (mc_i, mc) in enumerate(i.name for i in xp.f_model_class
                               if i != models.Ridge):
      df = imp.xs(metric, level="metric", axis=0) \
              .xs(mc, level="model", axis=1)
      bp = plt.boxplot(df.T, positions=positions[mc], widths=0.7)
      plt.setp(bp["boxes"], color=u.colors[mc_i+1])
      plt.setp(bp["whiskers"], color=u.colors[mc_i+1], lw=0.7)
      plt.setp(bp["caps"], color=u.colors[mc_i+1], lw=0.7)
      plt.setp(bp["medians"], color=u.colors[mc_i+1], lw=0.7)

   plt.axhline(y=1, color="black", lw=0.5)
   
   plt.xlim(0, 20)
   plt.xticks([2.5, 7.5, 12.5, 17.5],
              ["synthetic", "query string",
               "string vs. topic ridge", "query topic"])
   plt.tick_params(length=0, axis="x")

   if (metric_i == 1):
      plt.ylabel("times better than ridge")
   plt.ylim({"rmse":     (0,  5.0),
             "r2":       (0, 10.0),
             "hit_rate": (0,  2.5) }[metric])

   if (metric_i == 0):
      plt.plot([], color=u.colors[1], lw=1, label="threshold")
      plt.plot([], color=u.colors[2], lw=1, label="linear")
      plt.plot([], color=u.colors[3], lw=1, label="quadratic")
      plt.plot([], color=u.colors[4], lw=1, label="quartic")
      plt.legend(loc="upper right")

plt.tight_layout(h_pad=1)
plt.savefig("summary_boxplot.pdf")